# Comparison of Different Machine Learning Models

## Models Tried

1. RandomForestRegressor
    - Advantages: Why did we start with this? 
    - Drawbacks: Why did we pivot away?
1. K Nearest Neighbors
    - Advantages: Why was this the next choice?
    - Drawbacks: Why did we pivot away?
1. Gradient Boosting Classifier
    - Advantages: Why was this the next choice?
    - Drawbacks: Why did we pivot away?